In [ ]:
import os
import cv2
import random
import supervision
import numpy as np

In [ ]:
LEAF_IMAGES_DIR = ""
LEAF_MASKS_DIR = ""
BACKGROUND_IMAGES_DIR = ""
DATASET_DIR = ""
DATA_SUBDIR = "synthetic_leaf_instances"
TRAIN_IMAGES_DIR = "train/images"
TRAIN_IMAGES_SUBDIR = os.path.join(DATA_SUBDIR, TRAIN_IMAGES_DIR)
TRAIN_LABELS_DIR = "train/labels"
TRAIN_LABELS_SUBDIR = os.path.join(DATA_SUBDIR, TRAIN_LABELS_DIR)
VAL_IMAGES_DIR = "val/images"
VAL_IMAGES_SUBDIR = os.path.join(DATA_SUBDIR, VAL_IMAGES_DIR)
VAL_LABELS_DIR = "val/labels"
VAL_LABELS_SUBDIR = os.path.join(DATA_SUBDIR, VAL_LABELS_DIR)
DATASET_SPLIT = 0.8

In [ ]:
dataset = {
    "names": {
        0: "leaf"
    },
    "path": DATA_SUBDIR,
    "train": TRAIN_IMAGES_DIR,
    "val": VAL_IMAGES_DIR
}

In [ ]:
import yaml

# Define the output file path
output_file_path = os.path.join(DATASET_DIR, "dataset.yaml")

# Create the necessary directories if they don't exist
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

# Dump the dataset dictionary to a YAML file
with open(output_file_path, 'w') as file:
    yaml.dump(dataset, file, default_flow_style=False)

print(f"Dataset configuration saved to {output_file_path}")

In [ ]:
def transform_image(image, angle, scale, tx, ty):
    height, width = image.shape[:2]
    center = (width // 2, height // 2)

    # Rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, scale)
    # Translation matrix
    rotation_matrix[0, 2] += tx
    rotation_matrix[1, 2] += ty

    # Apply the transformation
    transformed_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return transformed_image

In [ ]:
def apply_mask(image, mask):
    return cv2.bitwise_and(image, image, mask=mask)

In [ ]:
def create_segmentation_mask(image, mask, value):
    segmented_mask = np.zeros_like(image)
    segmented_mask[mask > 0] = value
    return segmented_mask

In [ ]:
def create_yolo_label(segmentation_mask, image_width, image_height):
    yolo_labels = []
    contours, _ = cv2.findContours(segmentation_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if len(contour) < 3:
            continue
        x, y, w, h = cv2.boundingRect(contour)
        bbox = [x / image_width, y / image_height, w / image_width, h / image_height]
        polygon = contour.flatten().tolist()
        yolo_labels.append((0, bbox, polygon))
    return yolo_labels

In [ ]:
def process_image(image_files, mask_files, background_files, image_range, output_images, output_labels, image_index):
    # Randomly pick the number of images to combine from the provided range
    num_images = random.randint(image_range[0], image_range[1])

    # Randomly select `num_images` from the folder
    selected_indices = random.sample(range(len(image_files)), num_images)

    # Randomly select a background image
    background_index = random.randint(0, len(background_files) - 1)
    background_path = os.path.join(BACKGROUND_IMAGES_DIR, background_files[background_index])
    background_image = cv2.imread(background_path)

    # Check if the background image is loaded properly
    if background_image is None:
        print(f"Error: Background image {background_files[background_index]} not loaded properly")
        return

    # Initialize the combined image and segmentation mask
    combined_image = background_image.copy()
    segmentation_mask = np.zeros_like(background_image)
    yolo_labels = []

    for idx, i in enumerate(selected_indices):
        # Read image and corresponding mask
        image_path = os.path.join(LEAF_IMAGES_DIR, image_files[i])
        mask_path = os.path.join(LEAF_MASKS_DIR, mask_files[i])
        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Check if the images and masks are loaded properly
        if image is None or mask is None:
            print(f"Error: Image or mask {image_files[i]} not loaded properly")
            continue

        # Apply the mask to the image
        masked_image = apply_mask(image, mask)

        # Generate random transformations
        angle = random.uniform(-45, 45)  # Random rotation between -45 and 45 degrees
        scale = random.uniform(0.2, .7)  # Random scaling between 0.5 and 1.5
        tx = random.randint(-100, 100)  # Random translation in x direction
        ty = random.randint(-100, 100)  # Random translation in y direction

        # Apply transformations
        transformed_image = transform_image(masked_image, angle, scale, 0, 0)

        # Crop the transformed image to its extreme points that are not masked out
        y_indices, x_indices = np.where(mask > 0)
        if len(y_indices) == 0 or len(x_indices) == 0:
            continue
        x_min, x_max = x_indices.min(), x_indices.max()
        y_min, y_max = y_indices.min(), y_indices.max()
        cropped_transformed_image = transformed_image[y_min:y_max+1, x_min:x_max+1]

        # Generate random position to place the cropped transformed image on the background
        height, width = background_image.shape[:2]
        x_offset = random.randint(0, width - cropped_transformed_image.shape[1])
        y_offset = random.randint(0, height - cropped_transformed_image.shape[0])

        # Place the cropped transformed image on the background
        combined_image[y_offset:y_offset + cropped_transformed_image.shape[0], x_offset:x_offset + cropped_transformed_image.shape[1]] = cropped_transformed_image

        # Create the segmentation mask
        segmentation_value = [idx + 1, idx + 1, idx + 1]
        segmented_mask = create_segmentation_mask(masked_image, mask, segmentation_value)
        cropped_segmented_mask = segmented_mask[y_min:y_max+1, x_min:x_max+1]
        segmentation_mask[y_offset:y_offset + cropped_segmented_mask.shape[0], x_offset:x_offset + cropped_segmented_mask.shape[1]] = cropped_segmented_mask
        yolo_labels += create_yolo_label(segmentation_mask, image_width, image_height)

    # Get the dimensions of the combined image
    image_height, image_width = combined_image.shape[:2]

    # Save the combined image and YOLO labels
    combined_image_path = os.path.join(output_images, f"synthetic_{image_index}.png")
    cv2.imwrite(combined_image_path, combined_image)

    yolo_label_path = os.path.join(output_labels, f"synthetic_{image_index}.txt")
    with open(yolo_label_path, 'w') as f:
        for label in yolo_labels:
            class_id, bbox, polygon = label
            bbox_str = ' '.join(map(str, bbox))
            polygon_str = ' '.join(map(str, polygon))
            f.write(f"{class_id} {bbox_str} {polygon_str}\n")

In [ ]:
# Get the list of image and mask files
image_files = sorted([f for f in os.listdir(LEAF_IMAGES_DIR) if os.path.isfile(os.path.join(LEAF_IMAGES_DIR, f))])
mask_files = sorted([f for f in os.listdir(LEAF_MASKS_DIR) if os.path.isfile(os.path.join(LEAF_MASKS_DIR, f))])
background_files = sorted([f for f in os.listdir(BACKGROUND_IMAGES_DIR) if os.path.isfile(os.path.join(BACKGROUND_IMAGES_DIR, f))])

# Ensure the output folder exists
os.makedirs(DATA_SUBDIR, exist_ok=True)
os.makedirs(TRAIN_IMAGES_SUBDIR, exist_ok=True)
os.makedirs(TRAIN_LABELS_SUBDIR, exist_ok=True)
os.makedirs(VAL_IMAGES_SUBDIR, exist_ok=True)
os.makedirs(VAL_LABELS_SUBDIR, exist_ok=True)

# Test image

In [ ]:
import matplotlib.pyplot as plt
process_image(image_files, mask_files, background_files, [5, 10], "test/images", "test/labels", 0)
img = cv2.imread("test/images/synthetic_0.png")
plt.imshow(img)
plt.show()